In [1]:
# Dependencies and setup
import pandas as pd
import numpy as np
import requests
from pathlib import Path
import matplotlib.pyplot as plt

from api_keys import census_key

In [2]:
# Import unemployment rate csv file, create DataFrame, and set index to Area Type
unemployment_data = Path('resources/Local_Area_Unemployment_Statistics__LAUS___Annual_Average.csv')
unemployment_df = pd.read_csv(unemployment_data)
unemployment_df = unemployment_df.set_index('Area Type')
unemployment_df

# Filter Area Type to only include County and State data and reduce columns
area_type_df = unemployment_df.loc[['State','County'],
                                   ['Area Name', 'Year', 'Labor Force', 'Employment', 'Unemployment','Unemployment Rate']]
area_type_df

# Filter Year to only include 2017-2022 and filter out California totals
recent_unemployment = area_type_df.loc[(area_type_df['Year'] > 2016) & 
                          (area_type_df['Area Name'] != 'California')]

# Rename Area Name column
recent_unemployment = recent_unemployment.rename(columns = {'Area Name': 'County'})

recent_unemployment


,County,Year,Labor Force,Employment,Unemployment,Unemployment Rate
Area Type,,,,,,
County,Alameda County,2017,838200,807100,31100,3.7
County,Alpine County,2017,550,510,30,6.2
County,Amador County,2017,14690,13960,740,5.0
County,Butte County,2017,102000,96100,5900,5.8
County,Calaveras County,2017,21020,20010,1000,4.8
...,...,...,...,...,...,...
County,Tulare County,2022,207500,190500,16900,8.2
County,Tuolumne County,2022,19880,18990,890,4.5
County,Ventura County,2022,413600,398400,15200,3.7


In [3]:
# Reduce Columns
recent_unemployment = recent_unemployment[['County', 'Year', 'Unemployment Rate']]
recent_unemployment

,County,Year,Unemployment Rate
Area Type,,,
County,Alameda County,2017,3.7
County,Alpine County,2017,6.2
County,Amador County,2017,5.0
County,Butte County,2017,5.8
County,Calaveras County,2017,4.8
...,...,...,...
County,Tulare County,2022,8.2
County,Tuolumne County,2022,4.5
County,Ventura County,2022,3.7


In [4]:
# Import age csv file and create DataFrame
age_data = Path('resources/experiencing_homelessness_age_demographics.csv')
age_df = pd.read_csv(age_data)
age_df

,CALENDAR_YEAR,COC_ID,COC_NAME,AGE_GROUP_PUBLIC,EXPERIENCING_HOMELESSNESS_CNT
0,2017,All,California,18-24,15862
1,2017,All,California,25-34,28339
2,2017,All,California,35-44,25556
3,2017,All,California,45-54,27441
4,2017,All,California,55-64,23220
...,...,...,...,...,...
2136,2022,CA-614,San Luis Obispo County CoC,45-54,389
2137,2022,CA-614,San Luis Obispo County CoC,55-64,360
2138,2022,CA-614,San Luis Obispo County CoC,65+,174
2139,2022,CA-614,San Luis Obispo County CoC,Under 18,742


In [5]:
# Create clean DataFrame by dropping rows with '*' in count column
clean_age_df = age_df.loc[age_df['EXPERIENCING_HOMELESSNESS_CNT']!='*']
clean_age_df

,CALENDAR_YEAR,COC_ID,COC_NAME,AGE_GROUP_PUBLIC,EXPERIENCING_HOMELESSNESS_CNT
0,2017,All,California,18-24,15862
1,2017,All,California,25-34,28339
2,2017,All,California,35-44,25556
3,2017,All,California,45-54,27441
4,2017,All,California,55-64,23220
...,...,...,...,...,...
2136,2022,CA-614,San Luis Obispo County CoC,45-54,389
2137,2022,CA-614,San Luis Obispo County CoC,55-64,360
2138,2022,CA-614,San Luis Obispo County CoC,65+,174
2139,2022,CA-614,San Luis Obispo County CoC,Under 18,742


In [6]:
# Find all Continuums of Care in Los Angeles County
la_county_coc = clean_age_df.loc[clean_age_df["COC_NAME"].str.contains("Los Angeles County", case = False)]
la_county_coc

,CALENDAR_YEAR,COC_ID,COC_NAME,AGE_GROUP_PUBLIC,EXPERIENCING_HOMELESSNESS_CNT
244,2017,CA-600,Los Angeles County CoC,18-24,6040
245,2017,CA-600,Los Angeles County CoC,25-34,9236
246,2017,CA-600,Los Angeles County CoC,35-44,8201
247,2017,CA-600,Los Angeles County CoC,45-54,9235
248,2017,CA-600,Los Angeles County CoC,55-64,7642
...,...,...,...,...,...
2119,2022,CA-612,Glendale CoC (Los Angeles County),35-44,103
2120,2022,CA-612,Glendale CoC (Los Angeles County),45-54,113
2121,2022,CA-612,Glendale CoC (Los Angeles County),55-64,149
2122,2022,CA-612,Glendale CoC (Los Angeles County),65+,93


In [7]:
# Use replace to group all of the Los Angeles County CoCs
cocs_to_replace = ["Glendale CoC (Los Angeles County)",
                   "Pasadena CoC (Los Angeles County)", 
                   "Long Beach CoC (Los Angeles County)"
                  ]
clean_age_df['COC_NAME'] = clean_age_df['COC_NAME'].replace(cocs_to_replace, "Los Angeles County CoC")
clean_age_df.head()

C:\Users\mattcalvert\AppData\Local\Temp\ipykernel_11580\2570887098.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_age_df['COC_NAME'] = clean_age_df['COC_NAME'].replace(cocs_to_replace, "Los Angeles County CoC")


,CALENDAR_YEAR,COC_ID,COC_NAME,AGE_GROUP_PUBLIC,EXPERIENCING_HOMELESSNESS_CNT
0,2017,All,California,18-24,15862
1,2017,All,California,25-34,28339
2,2017,All,California,35-44,25556
3,2017,All,California,45-54,27441
4,2017,All,California,55-64,23220


In [8]:
# Change homeless count data type to integer
clean_age_df['EXPERIENCING_HOMELESSNESS_CNT'] = clean_age_df["EXPERIENCING_HOMELESSNESS_CNT"].astype(int)

# Group by year and COC Name and sum homeless counts
total_homeless_count = clean_age_df.groupby(['CALENDAR_YEAR', 'COC_NAME'])['EXPERIENCING_HOMELESSNESS_CNT'].sum().reset_index()
total_homeless_count

C:\Users\mattcalvert\AppData\Local\Temp\ipykernel_11580\2318218614.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_age_df['EXPERIENCING_HOMELESSNESS_CNT'] = clean_age_df["EXPERIENCING_HOMELESSNESS_CNT"].astype(int)


,CALENDAR_YEAR,COC_NAME,EXPERIENCING_HOMELESSNESS_CNT
0,2017,Alameda County CoC,6713
1,2017,"Alpine, Inyo, Mono Counties CoC",151
2,2017,"Amador, Calaveras, Mariposa, Tuolumne Counties...",368
3,2017,Butte County CoC,1400
4,2017,California,184234
...,...,...,...
245,2022,Stanislaus County CoC,6482
246,2022,Tehama County CoC,670
247,2022,Ventura County CoC,2919
248,2022,Yolo County CoC,1419


In [9]:
# Rename CALENDAR_YEAR column
total_homeless_count = total_homeless_count.rename(columns = {'CALENDAR_YEAR': 'Year'})
total_homeless_count

,Year,COC_NAME,EXPERIENCING_HOMELESSNESS_CNT
0,2017,Alameda County CoC,6713
1,2017,"Alpine, Inyo, Mono Counties CoC",151
2,2017,"Amador, Calaveras, Mariposa, Tuolumne Counties...",368
3,2017,Butte County CoC,1400
4,2017,California,184234
...,...,...,...
245,2022,Stanislaus County CoC,6482
246,2022,Tehama County CoC,670
247,2022,Ventura County CoC,2919
248,2022,Yolo County CoC,1419


In [10]:
# Import california population csv file and create DataFrame
population_data = Path('resources/california_population.csv')
population_df = pd.read_csv(population_data)
population_df.head()

,COUNTY,Year,Total Population,Total Housing Units,Single Detached,Single Attached,Two to Four,Five Plus,Mobile Homes,Occupied,Vacancy Rate,Persons per Household
0,Alameda,2017,"1,644,303","596,937","315,395","46,333","65,721","161,630","7,858","564,303",5.50%,2.84
1,Alpine,2017,"1,161","1,780","1,052",18,45,631,34,483,72.90%,2.35
2,Amador,2017,"36,900","18,189","14,905",570,605,690,"1,419","15,140",16.80%,2.18
3,Butte,2017,"225,468","98,871","61,689","3,112","8,976","10,855","14,239","90,791",8.20%,2.42
4,Calaveras,2017,"45,170","27,908","24,256",366,676,392,"2,218","18,371",34.20%,2.43


In [11]:
# Reduce columns to just county, year, and total population
population_df = population_df[['COUNTY','Year','Total Population']]

# Rename COUNTY column
population_df = population_df.rename(columns = {'COUNTY': 'County'})

#Filter out California
population_df = population_df.loc[population_df['County'] != 'California']

#add 'County' to the county names
population_df['County'] = population_df['County'] + ' County'

# Remove "," from Total Population
population_df['Total Population'] = population_df['Total Population'].str.replace(',','')

# Change Total Population data type to integer
population_df['Total Population'] = population_df['Total Population'].astype(int)
population_df

,County,Year,Total Population
0,Alameda County,2017,1644303
1,Alpine County,2017,1161
2,Amador County,2017,36900
3,Butte County,2017,225468
4,Calaveras County,2017,45170
...,...,...,...
348,Tulare County,2022,474507
349,Tuolumne County,2022,54465
350,Ventura County,2022,831533
351,Yolo County,2022,221639


In [12]:
# Import coc mapping csv file and create DataFrame
coc_data = Path('resources/County-CoC Mapping.csv')
coc_df = pd.read_csv(coc_data)

# Rename Continuum of Care column to COC_NAME
coc_df = coc_df.rename(columns = {'Continuum of Care': 'COC_NAME'})

coc_df.head()

,County,COC_NAME
0,Alameda County,Alameda County CoC
1,Alpine County,"Alpine, Inyo, Mono Counties CoC"
2,Inyo County,"Alpine, Inyo, Mono Counties CoC"
3,Mono County,"Alpine, Inyo, Mono Counties CoC"
4,Amador County,"Amador, Calaveras, Mariposa, Tuolumne Counties..."


In [13]:
# Merge population DF and coc DF on County
population_coc = pd.merge(population_df, coc_df, on='County',how = 'left')

# Group by year and COC Name and sum total population
population_coc_grouped = population_coc.groupby(['Year', 'COC_NAME'])['Total Population'].sum().reset_index()
population_coc_grouped

,Year,COC_NAME,Total Population
0,2017,Alameda County CoC,1644303
1,2017,"Alpine, Inyo, Mono Counties CoC",33350
2,2017,"Amador, Calaveras, Mariposa, Tuolumne Counties...",154922
3,2017,Butte County CoC,225468
4,2017,"Colusa, Glenn, Trinity Counties CoC",63889
...,...,...,...
235,2022,Stanislaus County CoC,548719
236,2022,Tehama County CoC,64907
237,2022,Ventura County CoC,831533
238,2022,Yolo County CoC,221639


In [15]:
# Merge recent unemployment DF and coc DF on county
unemployment_coc = pd.merge(recent_unemployment, coc_df, on='County',how = 'left')

# Group by year and COC Name and average unemployment rate
unemployment_coc_grouped = unemployment_coc.groupby(['Year', 'COC_NAME'])['Unemployment Rate'].mean().reset_index()
unemployment_coc_grouped

,Year,COC_NAME,Unemployment Rate
0,2017,Alameda County CoC,3.700000
1,2017,"Alpine, Inyo, Mono Counties CoC",5.033333
2,2017,"Amador, Calaveras, Mariposa, Tuolumne Counties...",5.275000
3,2017,Butte County CoC,5.800000
4,2017,"Colusa, Glenn, Trinity Counties CoC",9.333333
...,...,...,...
235,2022,Stanislaus County CoC,5.400000
236,2022,Tehama County CoC,4.800000
237,2022,Ventura County CoC,3.700000
238,2022,Yolo County CoC,4.000000


In [17]:
# Merge pop_coc_reduced and total_homeless_count
total_pop_homeless = pd.merge(population_coc_grouped, total_homeless_count, on=['COC_NAME','Year'],how = 'left')

# Reorder columns
total_pop_homeless = total_pop_homeless[['COC_NAME', 'Year', 'Total Population', 'EXPERIENCING_HOMELESSNESS_CNT']]
total_pop_homeless

,COC_NAME,Year,Total Population,EXPERIENCING_HOMELESSNESS_CNT
0,Alameda County CoC,2017,1644303,6713.0
1,"Alpine, Inyo, Mono Counties CoC",2017,33350,151.0
2,"Amador, Calaveras, Mariposa, Tuolumne Counties...",2017,154922,368.0
3,Butte County CoC,2017,225468,1400.0
4,"Colusa, Glenn, Trinity Counties CoC",2017,63889,190.0
...,...,...,...,...
235,Stanislaus County CoC,2022,548719,6482.0
236,Tehama County CoC,2022,64907,670.0
237,Ventura County CoC,2022,831533,2919.0
238,Yolo County CoC,2022,221639,1419.0
